In [ ]:
import math

import pandas as pd

import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score




# Read data from CSV

data = pd.read_csv("/content/sample_data/heart.csv")




# Preprocess the data if needed (e.g., encoding categorical variables)

# Assuming no preprocessing needed in this case




# Split data into input features (X) and target variable (Y)

X = data.drop("target", axis=1)

Y = data["target"]




# Standardize input features

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)




# Perform K-Means clustering

K_cent = 8

km = KMeans(n_clusters=K_cent, max_iter=100)

km.fit(X_scaled)

cent = km.cluster_centers_




# Calculate sigma

max_dist = 0

for i in range(K_cent):

    for j in range(K_cent):

        dist = np.linalg.norm(cent[i] - cent[j])

        if dist > max_dist:

            max_dist = dist

sigma = max_dist / math.sqrt(2 * K_cent)




# Compute RBF transformation matrix for training data

row, column = X_scaled.shape

G = np.empty((row, K_cent), dtype=float)

for i in range(row):

    for j in range(K_cent):

        dist = np.linalg.norm(X_scaled[i] - cent[j])

        G[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))




# Compute weights

GTG = np.dot(G.T, G)

GTG_inv = np.linalg.inv(GTG)

fac = np.dot(GTG_inv, G.T)

W = np.dot(fac, Y)




# Split data into training and testing sets

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.33, random_state=42)




# Compute RBF transformation matrix for testing data

row_test, _ = X_test.shape

G_test = np.empty((row_test, K_cent), dtype=float)

for i in range(row_test):

    for j in range(K_cent):

        dist = np.linalg.norm(X_test[i] - cent[j])

        G_test[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))




# Predict output

prediction = np.dot(G_test, W)

prediction = 0.5 * (np.sign(prediction - 0.5) + 1)




# Evaluate model

score = accuracy_score(prediction, Y_test)

print("Accuracy:", score)
